<a href="https://colab.research.google.com/github/Inaldo-Barros/Assistencia_do_zero/blob/main/Sistema_Virtual_do_zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install speechrecognition gtts pywhatkit
!pip install pyaudio  # Para capturar áudio do microfone (pode precisar de instalação adicional no seu ambiente)
!pip install geopy    # Para encontrar a localização da farmácia mais próxima


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.54-py3-none-any.whl size=37637 sha256=c4a3b3147fe014214d838ddf3d0683461e5a01867767ea661500

In [36]:
!pip install pywhatkit

Vamos importar as bibliotecas necessárias:

In [40]:
import speech_recognition as sr
from gtts import gTTS
import webbrowser
import os
from geopy.geocoders import Nominatim
from geopy.distance import geodesic


Criaremos uma função para transformar texto em áudio usando gtts:

In [32]:
def text_to_speech(text):
    tts = gTTS(text=text, lang='pt')
    tts.save('response.mp3')
    os.system('mpg123 response.mp3')


Usaremos speech_recognition para capturar e transcrever a fala:

In [33]:
def speech_to_text():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Diga algo:")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio, language='pt-BR')
            print(f"Você disse: {text}")
            return text.lower()
        except sr.UnknownValueError:
            print("Não consegui entender o que você disse.")
            return ""
        except sr.RequestError:
            print("Não consegui acessar o serviço de reconhecimento de fala.")
            return ""


Adicionaremos funções para abrir o Wikipedia, abrir o YouTube e encontrar a farmácia mais próxima:

In [34]:
def open_wikipedia(query):
    url = f"https://pt.wikipedia.org/wiki/{query.replace(' ', '_')}"
    webbrowser.open(url)
    text_to_speech(f"Abrindo o Wikipedia para {query}")

def open_youtube(query):
    pywhatkit.playonyt(query)
    text_to_speech(f"Abrindo o YouTube para {query}")

def find_nearest_pharmacy():
    geolocator = Nominatim(user_agent="assistant")
    location = geolocator.geocode("Caaporã, Paraíba, Brasil")
    pharmacies = geolocator.geocode("pharmacy", exactly_one=False)

    nearest_pharmacy = None
    min_distance = float('inf')

    for pharmacy in pharmacies:
        distance = geodesic((location.latitude, location.longitude), (pharmacy.latitude, pharmacy.longitude)).km
        if distance < min_distance:
            min_distance = distance
            nearest_pharmacy = pharmacy

    if nearest_pharmacy:
        text_to_speech(f"A farmácia mais próxima está a {min_distance:.2f} quilômetros em {nearest_pharmacy.address}")
    else:
        text_to_speech("Não consegui encontrar uma farmácia próxima.")


Por fim, integraremos tudo em um sistema que responde a comandos de voz:

In [ ]:
def main():
    while True:
        command = speech_to_text()
        if 'wikipedia' in command:
            query = command.replace('wikipedia', '').strip()
            open_wikipedia(query)
        elif 'youtube' in command:
            query = command.replace('youtube', '').strip()
            open_youtube(query)
        elif 'farmácia mais próxima' in command:
            find_nearest_pharmacy()
        elif 'sair' in command:
            text_to_speech("Encerrando o assistente virtual. Até logo!")
            break
        else:
            text_to_speech("Comando não reconhecido. Por favor, tente novamente.")

if __name__ == "__main__":
    main()
